In [7]:
import pandas as pd
import numpy as np

In [8]:
#passengers per stop pay day of week 2020 using CSV
import sys
import psycopg2
import pickle
import csv


#==============================================================================
# Main function
#==============================================================================
def main(argv):
    tbl = 'tridelta'
    fname = 'tridelta_stops_by_passenger_2019.pickle'
    conn = psycopg2.connect("dbname=transit user=postgres", password="postgres")
    cur = conn.cursor()
    week = ['0', '1', '2', '3', '4', '5', '6']

    dow_passenger=[]
    dow_2020=[]
    stops_2020=[]
    with open("tridelta_stops_by_passenger_2019_off.csv",'w', newline='') as f:
        writer = csv.writer(f, dialect='excel')
        writer.writerow(['Date', 'Stop ID', 'Passengers'])
        for dow in week:
            stmt = "SELECT DATE(act_stop_time), stop_id, sum(psgr_off) as passengers from "
            stmt += tbl
            stmt += " WHERE EXTRACT(DOW FROM act_stop_time) = " + dow
            stmt += " AND act_stop_time >= TO_DATE('2019-01-01', 'YYYY-MM-DD') AND act_stop_time <=  TO_DATE('2019-12-31', 'YYYY-MM-DD')"
            stmt += " GROUP BY DATE(act_stop_time), stop_id "
            stmt += " ORDER BY DATE(act_stop_time), passengers;"

            cur.execute(stmt)

            result = cur.fetchall()
            date=[]
            stop_id=[]
            passenger=[]
            for record in result:
                writer.writerow([record[0], record[1], record[2]])

    conn.commit()
    cur.close()
    conn.close()


if __name__ == "__main__":
    main(sys.argv)